# Lab 7: Human motion generation

## Advanced deep learning

## Setup

### Setup dataset


Please access this Google Drive folder: [link](https://drive.google.com/drive/folders/1V5yzlwBPSNVPj33SfDHnvMykXISh3CyB?usp=sharing) and create a shortcut in the root of your Google Drive `/content/drive/MyDrive/`.

In [5]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/MyDrive/humanml3d-data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
caption_clip  caption_raw  checkpoints	humanml3d_test_split.txt  smplh  smpl_rifke


### Setup environment

Make sure you're running on a T4 GPU Colab instance; if not, activate it.

In [6]:
!nvidia-smi

Tue Mar  4 17:40:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
import torch
torch.cuda.is_available()

True

Clone the lab repository.

In [8]:
!git clone https://github.com/robincourant/lab7-CSC52087EP.git
#!git clone https://github.com/robincourant/lab-MotionDiT.git

Cloning into 'lab7-CSC52087EP'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 78 (delta 5), reused 78 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (78/78), 1.64 MiB | 28.93 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [9]:
%cd lab7-CSC52087EP
#%cd lab-MotionDiT
!ln -s /content/drive/MyDrive/humanml3d-data ./ # Plug the dataset in the repo

/content/lab7-CSC52087EP


Install required libraries

In [10]:
!pip install hydra-core
!pip install pyrender
!pip install smplx
!pip install torchtyping
!pip install lightning
!pip install ema_pytorch

## Human motion dataset and representation

### HumanML3D dataset

#### Question 1: How many motions and descriptions are included in the HumanML3D dataset, and what are their average lengths?
*Answer*

It contains 14,616 motion sequences and 44,970 text descriptions. On average, each motion lasts around 3 - 3.5 seconds (roughly 200 frames at 60 fps), and each text description is about 15 words long.

### SMPL representation


#### Question 2: What are the different input parameters of the SMPL model used to infer the vertices, and what do they represent?
*Answer*

The SMPL model uses:

- ```betas: Optional[Tensor] = None``` Shape coefficients (typically 10 values) that determine the body's overall shape.
- ```global_orient: Optional[Tensor] = None```
 A 3D rotation (often in axis - angle form) that sets the overall body orientation.
- ```body_pose: Optional[Tensor] = None``` Joint rotations (for 23 joints) that define the articulated pose.
- ```transl: Optional[Tensor] = None``` A 3D vector that positions the body in space.

#### Code 2
*Complete `visualize_smpl.py`*

In [12]:

!HYDRA_FULL_ERROR=1 PYTHONPATH=$(pwd) python src/visualize_smpl.py

import moviepy.editor

moviepy.editor.ipython_display("./smpl.mp4")

[2025-03-04 17:42:24,954][numexpr.utils][INFO] - NumExpr defaulting to 2 threads.
/content/lab7-CSC52087EP/src/datasets/modalities/human_dataset.py:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this

## Model architectures

### Config A: Incontext

#### Code 3
*Complete `src/models/modules/incontext.py`*

In [58]:
!PYTHONPATH=$(pwd) python src/models/modules/incontext.py

Test passed!


### Config B: AdaLN

#### Question 3:
*Answer*
Here is the AdaLN operation in LaTeX:

$
\operatorname{AdaLN}(x,\gamma,\beta) = \gamma \odot \frac{x - \mu(x)}{\sqrt{\sigma^2(x) + \epsilon}} + \beta
$

where:
- $ \mu(x) $ is the mean of $ x $,
- $ \sigma^2(x) $ is the variance of $ x $,
- $ \epsilon $ is a small constant for numerical stability,
- $ \gamma $ and $ \beta $ are the shift and scale parameters,
- $ \odot $ denotes element-wise multiplication.

#### Code 4
*Complete `src/models/modules/adaln.py`*

In [59]:
!PYTHONPATH=$(pwd) python src/models/modules/adaln.py

Test passed!


### Config C: Cross attention

#### Question 4:
*Answer*
Below is the cross-attention operation in LaTeX, where $ W_Q $, $ W_K $, and $ W_V $ are the query, key, and value projection matrices, respectively, and $ d $ is the token dimension:

$
\text{CA}(x,c) = \operatorname{softmax}\!\left(\frac{x\,W_Q\,(c\,W_K)^\top}{\sqrt{d}}\right) \,(c\,W_V)
$

In this expression:
- $ x $ is the input token sequence,
- $ c $ is the conditioning token sequence,
- $ x\,W_Q $ computes the queries,
- $ c\,W_K $ computes the keys,
- $ c\,W_V $ computes the values,
- The division by $\sqrt{d}$ scales the dot products for numerical stability, and
- The softmax normalizes the attention scores before multiplying with the values.

#### Code 5
*Complete `src/models/modules/cross attention.py`*

In [60]:
!PYTHONPATH=$(pwd) python src/models/modules/cross_attention.py

Test passed!


#### Question 5:
*Answer*
AdaLN injects conditioning by directly modulating normalized features. It computes:

$
\operatorname{AdaLN}(x,\gamma,\beta) = \gamma \odot \frac{x - \mu(x)}{\sqrt{\sigma^2(x)+\epsilon}} + \beta,
$

where $\gamma$ and $\beta$ come from the conditioning signal. This means the conditioning continuously influences every layer without needing extra layers to compute queries, keys, and values as in cross-attention. As a result, AdaLN is more efficient and stable, using fewer parameters while ensuring consistent conditioning across layers.

## Diffusion framework

### DDPM

#### Code 6
*Complete `src/training/losses/ddpm.py`*

In [61]:
!PYTHONPATH=$(pwd) python src/training/losses/ddpm.py

Test passed!


#### Code 7
*Complete `src/training/sampler/ddpm.py`*

In [62]:
!PYTHONPATH=$(pwd) python src/generate.py batch_size=1 diffuser/sampler@diffuser.test_sampler=ddpm seed=2

import moviepy.editor
moviepy.editor.ipython_display("./generation_ddpm_incontext.mp4")

Seed set to 2
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
/content/lab7-CSC52087EP/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where

### DDIM

#### Question 6:
*Answer*
#### DDPM Reverse Process (Markovian):
$
x_{t-1} \sim \mathcal{N}\Bigl(\mu_\theta(x_t, t),\, \Sigma_\theta(x_t, t)\Bigr)
$
In DDPM, the reverse process samples $x_{t-1}$ from a Gaussian distribution, where the mean $\mu_\theta(x_t, t)$ and covariance $\Sigma_\theta(x_t, t)$ are predicted by the model. This makes the process stochastic and Markovian, as each step depends only on the previous state $x_t$ and involves randomness.


#### DDIM Reverse Process (Non-Markovian/Deterministic):
$
x_t = \sqrt{\bar{\alpha}_t}\, x_0 + \sqrt{1-\bar{\alpha}_t}\,\epsilon
$
Rearranging for a deterministic reverse mapping, DDIM uses:
$
x_{t-1} = \sqrt{\bar{\alpha}_{t-1}}\, x_0 + \sqrt{1-\bar{\alpha}_{t-1}}\,\epsilon,
$
which directly computes $x_{t-1}$ from the predicted clean sample $x_0$ and the noise $\epsilon$. This formulation is non-Markovian because it relies on the forward process's closed-form expression rather than sampling from a learned distribution at every step.


DDPM samples $x_{t-1}$ via a Gaussian:
$
x_{t-1} \sim \mathcal{N}\Bigl(\mu_\theta(x_t, t),\, \Sigma_\theta(x_t, t)\Bigr),
$
introducing noise at each step, while DDIM computes a deterministic mapping:
$
x_{t-1} = \sqrt{\bar{\alpha}_{t-1}}\, x_0 + \sqrt{1-\bar{\alpha}_{t-1}}\,\epsilon,
$
which enables faster sampling by skipping intermediate stochastic steps.


#### Code 8
*Complete `src/training/sampler/ddim.py`*

In [63]:
!PYTHONPATH=$(pwd) python src/generate.py batch_size=1 diffuser/sampler@diffuser.test_sampler=ddim seed=3

import moviepy.editor
moviepy.editor.ipython_display("./generation_ddim_incontext.mp4")

Seed set to 3
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
/content/lab7-CSC52087EP/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where

#### Question 7:
*Answer*

Yes. DDPM adds random noise at every step, so it tends to produce more varied outputs. DDIM, on the other hand, can be made deterministic (or partially so), which can give sharper results but less variety.

## Result analysis

### Qualitative analysis

#### Code 9

In [64]:
!PYTHONPATH=$(pwd) python src/generate.py batch_size=1 diffuser/network=incontext \
checkpoint_path=./humanml3d-data/checkpoints/incontext.ckpt

import moviepy.editor
moviepy.editor.ipython_display("./generation_ddpm_incontext.mp4")

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
/content/lab7-CSC52087EP/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case wher

In [65]:
!PYTHONPATH=$(pwd) python src/generate.py batch_size=1 diffuser/network=adaln \
checkpoint_path=./humanml3d-data/checkpoints/adaln.ckpt

import moviepy.editor
moviepy.editor.ipython_display("./generation_ddpm_adaln.mp4")

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
/content/lab7-CSC52087EP/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case wher

In [66]:
!PYTHONPATH=$(pwd) python src/generate.py batch_size=1 diffuser/network=cross_attention \
checkpoint_path=./humanml3d-data/checkpoints/cross_attention.ckpt

import moviepy.editor
moviepy.editor.ipython_display("./generation_ddpm_cross_attention.mp4")

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
/content/lab7-CSC52087EP/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case wher

#### Question 8
*Answer*

Yes, there are clear qualitative differences. For the caption "a man kicks something or someone with his left leg":

- The **InContext** model produces a motion with one clean, distinct kick.
- The **AdaLN** model generates a motion with two kicks, suggesting a slight over-repetition.
- The **Cross-Attention** model produces nearly three kicks but with noticeable noise and instability.

This implies that the way conditioning is integrated (InContext vs. AdaLN vs. Cross-Attention) significantly affects the timing and clarity of the generated motions.

### Quantitative analysis

#### Question 9:
*Answer*

The main assumption is that both the reference and generated feature spaces can be approximated as multivariate Gaussian distributions. This is important because it lets us compare two complex, high-dimensional distributions using only their first two moments—the mean and covariance.

Mathematically, we assume:

$
\text{Reference features} \sim \mathcal{N}(\mu_r, \Sigma_r) \quad \text{and} \quad \text{Generated features} \sim \mathcal{N}(\mu_g, \Sigma_g).
$

Because of this Gaussian assumption, the Fréchet Distance (often used as the FID score) is given by the closed-form expression:

$
\text{FID} = \|\mu_r - \mu_g\|^2 + \operatorname{Tr}\Bigl(\Sigma_r + \Sigma_g - 2\bigl(\Sigma_r \Sigma_g\bigr)^{1/2}\Bigr).
$

**Reasoning with Math:**

- **Moment Matching:** For Gaussian distributions, knowing the mean $\mu$ and covariance $\Sigma$ completely specifies the distribution. This means we only need to compare $\mu_r$ with $\mu_g$ and $\Sigma_r$ with $\Sigma_g$ to assess the similarity between the two distributions.

- **2-Wasserstein Distance:** The closed-form for the squared 2-Wasserstein distance between two Gaussians is derived as:

  $
  W_2^2(\mathcal{N}(\mu_r,\Sigma_r),\mathcal{N}(\mu_g,\Sigma_g)) = \|\mu_r - \mu_g\|^2 + \operatorname{Tr}\left(\Sigma_r + \Sigma_g - 2\left(\Sigma_r^{1/2}\Sigma_g\Sigma_r^{1/2}\right)^{1/2}\right).
  $

  In practice, a simplified form is used, which is equivalent under the Gaussian assumption.

- **Simplification:** By assuming Gaussianity, we avoid the need to compare every detail of the feature distributions. Instead, the difference between the two distributions is fully captured by differences in their means and covariances. This greatly simplifies the calculation and still provides a meaningful metric for evaluating the quality of generated samples.

In summary, the Gaussian assumption reduces the complex problem of comparing high-dimensional distributions to a comparison of their means and covariances, which is why the Fréchet Distance can be computed efficiently in practice.

#### Code 10
*Complete src/metrics/frechet.py*

#### Code 11
*Complete src/metrics/similarity.py*

#### Code 12

#### Bonus 1:
*Answer here*

- **R1, R2, R3**: These check if the correct motion is in the top 1, 2, or 3 results for each text query.  
- **PRDC**: Precision, Recall, Density, and Coverage, which compare how well the generated motions match and cover the real motions, reflecting both quality (precision) and diversity (recall, density, coverage).

In [67]:
!PYTHONPATH=$(pwd) python src/evaluate.py diffuser/network=incontext \
checkpoint_path=./humanml3d-data/checkpoints/incontext.ckpt

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
[2025-03-04 20:43:54,431][OpenGL.acceleratesupport][INFO] - No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'
/content/lab7-CSC52087EP/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlis

In [68]:
!PYTHONPATH=$(pwd) python src/evaluate.py diffuser/network=adaln \
checkpoint_path=./humanml3d-data/checkpoints/adaln.ckpt

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
[2025-03-04 20:51:46,201][OpenGL.acceleratesupport][INFO] - No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'
/content/lab7-CSC52087EP/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlis

In [69]:
!PYTHONPATH=$(pwd) python src/evaluate.py diffuser/network=cross_attention \
checkpoint_path=./humanml3d-data/checkpoints/cross_attention.ckpt

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
[2025-03-04 20:59:58,925][OpenGL.acceleratesupport][INFO] - No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'
/content/lab7-CSC52087EP/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlis

#### Question 10:
*Answer*

Based on the metrics:

- **Adaln**: With the lowest FID (54.66), it appears to produce the most realistic and natural motions.
- **Cross-Attention**: Although its FID is higher (91.47), it scores best on TMR (0.6487) and retrieval (R1, R2, R3), suggesting superior alignment between the generated motions and the text descriptions.
- **InContext**: With the highest FID (132.57) and lower retrieval scores, it seems to generate less natural motions with weaker text-motion coherence.

The trade-off is: adaln excels in motion realism, while cross-attention better captures text-motion correspondence.

#### Bonus 2:
*Answer*

Using only 10 x 64 samples is a small evaluation set, so the metrics especially FDTMR, which depends on accurately estimating means and covariances can be noisy and unreliable. To improve reliability, we should use more samples, run multiple evaluations and average the results, and possibly report confidence intervals.

In [74]:
!zip -r /content/project.zip /content/lab7-CSC52087EP -x "/content/lab7-CSC52087EP/humanml3d-data/*"

  adding: content/lab7-CSC52087EP/ (stored 0%)
  adding: content/lab7-CSC52087EP/lab7_CSC52087EP.pdf (deflated 29%)
  adding: content/lab7-CSC52087EP/README.md (deflated 18%)
  adding: content/lab7-CSC52087EP/generation_ddpm_adaln.mp4 (deflated 1%)
  adding: content/lab7-CSC52087EP/generation_ddpm_cross_attention.mp4 (deflated 1%)
  adding: content/lab7-CSC52087EP/configs/ (stored 0%)
  adding: content/lab7-CSC52087EP/configs/dataset/ (stored 0%)
  adding: content/lab7-CSC52087EP/configs/dataset/humanml3d.yaml (deflated 75%)
  adding: content/lab7-CSC52087EP/configs/dataset/standardization/ (stored 0%)
  adding: content/lab7-CSC52087EP/configs/dataset/standardization/tmr-rifke-std.pt (deflated 34%)
  adding: content/lab7-CSC52087EP/configs/dataset/standardization/tmr-rifke-mean.pt (deflated 32%)
  adding: content/lab7-CSC52087EP/configs/dataset/standardization/rifke-mean.pt (deflated 32%)
  adding: content/lab7-CSC52087EP/configs/dataset/standardization/rifke-std.pt (deflated 35%)
  ad